In [1]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import OpenAI


assert load_dotenv(), "Failed to load .env file"

In [2]:
# import sys
# sys.path.append("/home/baris/repos/rlvr")

# from scripts.musique import evaluate

In [ ]:
model = "Qwen/Qwen2.5-7B-Instruct"
datasets_str = "bdsaglam/musique-mini,answerable,validation[:8]"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.1
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")
client = OpenAI()

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-7B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation[:8] (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.1
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 8/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

2025-08-24 19:18:29 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 8 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Running 8 rollouts:   0%|          | 0/8 [00:00<?, ?it/s]2025-08-24 19:18:29 - verifiers.envs.MuSiQueEnv - ERROR - Error getting model response: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}} 

Exiting...
Running 8 rollouts:   0%|          | 0/8 [00:00<?, ?it/s]
2025-08-24 19:18:29 - verifiers.envs.MuSiQueEnv - ERROR - Error getting model response: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}} 

Exiting...
2025-08-24 19:18:29 - verifiers.envs.MuSiQueEnv - ERROR - Error getting model response: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}} 

Exiting...


NotFoundError: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}}

2025-08-24 19:18:30 - verifiers.envs.MuSiQueEnv - ERROR - Error getting model response: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}} 

Exiting...
2025-08-24 19:18:31 - verifiers.envs.MuSiQueEnv - ERROR - Error getting model response: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}} 

Exiting...
2025-08-24 19:18:31 - verifiers.envs.MuSiQueEnv - ERROR - Error getting model response: Error code: 404 - {'error': {'message': 'The model `Qwen/Qwen2.5-7B-Instruct` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}} 

Exiting...


In [ ]:
len(results.prompt), len(results.completion), len(results.answer), len(results.state), len(results.info), len(results.task), len(results.reward)

(8, 8, 8, 8, 8, 8, 8)

In [ ]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [ ]:
from verifiers.types import GenerateOutputs

for field in GenerateOutputs.__fields__.keys():
    print(field)

    _type = type(getattr(results, field))
    if _type == list:
        print(getattr(results, field)[0])
    else:
        print(type(getattr(results, field)))

prompt
[{'content': "Answer the question based on the information provided by tools. You have access to the following tools:\n====\n{tool_descriptions}\n====\n\nFor each step:\n1. Think through your reasoning inside <think> tags\n2. Use tools to retrieve relevant documents\n3. Continue until you have found the answer through multi-hop reasoning\n4. In the **last** step:\n   - Reflect on your previous steps inside <think> tags\n   - Cite the documents you used inside <cite> tags by their IDs, e.g. `<cite>1, 2, 3</cite>`\n   - Give your final answer inside <answer> tags\nAn example for your final message:\n```\n<think>\n[your thinking here]\n</think> \n<cite>\n[IDs of the documents that back your answer, e.g. 5, 3, 2]\n</cite>\n<answer>\n[your final answer here without any additional text]\n</answer>\n```\n\n- Do not make up tools or arguments that aren't listed.\n- Questions require multi-hop reasoning across multiple documents.\n- Continue searching until you find all relevant informat

/tmp/ipykernel_1052597/3696243233.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for field in GenerateOutputs.__fields__.keys():


In [ ]:
ds = vf_env.make_dataset(results)
ds


Dataset({
    features: ['prompt', 'completion', 'answer', 'info', 'task', 'reward', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 8
})

In [ ]:
ds.to_json("../outputs/sample.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

255698